# Desafio - Santander Dev Week 2023 - ETL com Pyton

# Instalações Iniciais

In [ ]:
!pip install yfinance --upgrade --no-cache-dir
!pip install pandas --upgrade --no-cache-dir
!pip install --upgrade xlrd

# Extract

In [ ]:
import pandas as pd
import yfinance as yf

# Criar lista de ativos, ou baixar no link abaixo:
# baixar em https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-brasil-100-ibrx-100-composicao-da-carteira.htm
# abrir arquivo, tirar primeira linha e as duas ultimas, salvar com excel (xls) com nome ativos

symbol_list_ibrx = list(pd.read_excel('ativos.xls')['Código'].values) #copia todos os códigos dos ativos para a variável
symbol_list_ibrx

# Transform

In [25]:
df_resultado = pd.DataFrame()
for ativo in symbol_list_ibrx: #para cada ativo da variável com a lista de ativos fazer:
    df_info_ativo =pd.DataFrame()
    chamada_api = yf.Ticker(ativo+'.SA') #baixa as informações do ativo selecionado
    closes = chamada_api.history(period='36mo')['Close'].values #busca o preço de fechamento e grava na variável closes
    dividendos = chamada_api.history(period='36mo')['Dividends'].values #busca os dividendos e grava na variável dividendos
    datas = list(chamada_api.history(period='36mo').index) #busca as datas e grava na variável datas


    df_info_ativo['Closes'] = closes #grava a informação no dataframe
    df_info_ativo['FechamentoDiaAnterior'] = df_info_ativo['Closes'].shift()  #pega o fechamento do dia anterior (que está na linha anterior)
    df_info_ativo['Dividendos'] = dividendos
    df_info_ativo['Datas'] = datas
    df_info_ativo['DivPorcent'] = (df_info_ativo['Dividendos'] / df_info_ativo['Closes'])*100
    df_info_ativo = df_info_ativo.set_index('Datas',drop = True) #coloca a coluna datas como index e limpa os indexs que não tem informação


    soma_dividendos = 0 #criação da variável que vai registrar os dividendos

    for data,dados in df_info_ativo.iterrows():
        if data.year == 2020 or data.year == 2021: #filtra por anos determinados
            soma_dividendos += dados['DivPorcent'] #soma a coluna dividendos dos anos selecionados

    dividend_yield = (soma_dividendos/2)  #faz a média dos dividendos e divide pelo último preço de fechamento do ativo / depois divide por 100 para aparecer como porcentagem

    df_resultado[ativo] = [dividend_yield]

#MOSTRARÁ A PORCENTAGEM PAGA EM RELAÇÃO AO SEU PRÓPRIO PREÇO.
df_resultado_final = df_resultado.transpose().sort_values(by = 0,ascending=False)[:20] #mostra os 20 melhores / ascending=false (lista das maiores para as menores)


In [ ]:
df_resultado_final

# Load - Exportar para XLS

In [29]:
nome_do_arquivo_excel = 'melhores_pagadores_de_dividendos.xlsx'


# Exporta o DataFrame para o arquivo Excel
df_resultado_final.to_excel(nome_do_arquivo_excel, index=True)

print(f'DataFrame exportado para {nome_do_arquivo_excel} com sucesso.')

DataFrame exportado para melhores_pagadores_de_dividendos.xlsx com sucesso.
